In [39]:
# Design MapReduce algorithms to take a very large file of integers and produce as output:
#(a) The largest integer.
#(b) The average of all the integers.
#(c) The same set of integers, but with each integer appearing only once.
#(d) The count of the number of distinct integers in the input.

from pyspark import SparkContext, SparkConf

sc = SparkContext("local", "Word Count")
txt = sc.textFile('numbers.txt') # list of line of sentences
words = txt.flatMap(lambda line: line.split()) # transforming sentences into separate words
numbers = [int(word) for word in words.collect()]
rdd = sc.parallelize(numbers)
print("maximum number =" , rdd.max())
print("average number =" , rdd.mean())
print("distinct numbers =" , rdd.distinct().collect())
print("count distinct numbers =" , len(rdd.distinct().collect()) )
sc.stop()

maximum number = 55
average number = 8.875
distinct numbers = [1, 2, 5, 7, 8, 11, 55, 9, 10, 3]
count distinct numbers = 10


In [38]:
sc.stop()

In [112]:
# Build a Language Model 
#Count the number of times each n-word sequence occurs in a given input file (book of Arthur Conan Doyle).
#Find Top-10 results to report for the cases when n = 2,4,5. 
#Can you find the most occurring/repeated name of the person? Is it Holmes or Watson? 

def n_grams(n , line ):
    n_words = []
    line = line.split()
    for w in range(0, len(line) - n +1 ):
        word = []
        for j in range(0, n):
            word.append(line[w+j])
        n_words.append(word)

    return n_words

In [113]:
sc.stop()

In [114]:
from pyspark import SparkContext, SparkConf

sc = SparkContext("local", "Word Count")
txt = sc.textFile('Arthur.txt') # list of line of sentences
#n = 5 # n-gram
n = 4 # n-gram
#n = 2 # n-gram
words = txt.flatMap(lambda line: n_grams(n , line)) # transforming sentences into separate words



In [115]:
words.collect()

[['Project', "Gutenberg's", 'The', 'Adventures'],
 ["Gutenberg's", 'The', 'Adventures', 'of'],
 ['The', 'Adventures', 'of', 'Sherlock'],
 ['Adventures', 'of', 'Sherlock', 'Holmes,'],
 ['of', 'Sherlock', 'Holmes,', 'by'],
 ['Sherlock', 'Holmes,', 'by', 'Arthur'],
 ['Holmes,', 'by', 'Arthur', 'Conan'],
 ['by', 'Arthur', 'Conan', 'Doyle'],
 ['This', 'eBook', 'is', 'for'],
 ['eBook', 'is', 'for', 'the'],
 ['is', 'for', 'the', 'use'],
 ['for', 'the', 'use', 'of'],
 ['the', 'use', 'of', 'anyone'],
 ['use', 'of', 'anyone', 'anywhere'],
 ['of', 'anyone', 'anywhere', 'at'],
 ['anyone', 'anywhere', 'at', 'no'],
 ['anywhere', 'at', 'no', 'cost'],
 ['at', 'no', 'cost', 'and'],
 ['no', 'cost', 'and', 'with'],
 ['almost', 'no', 'restrictions', 'whatsoever.'],
 ['no', 'restrictions', 'whatsoever.', 'You'],
 ['restrictions', 'whatsoever.', 'You', 'may'],
 ['whatsoever.', 'You', 'may', 'copy'],
 ['You', 'may', 'copy', 'it,'],
 ['may', 'copy', 'it,', 'give'],
 ['copy', 'it,', 'give', 'it'],
 ['it,', 'gi

In [116]:
# words_list = words.collect()
# type(words_list)
#words.collect()
rdd_nwords = sc.parallelize(words.collect())
list_n_words = rdd_nwords.map(lambda word: (tuple(word), 1))
list_n_words.collect()
list_n_words_groups = list_n_words.reduceByKey(lambda a, b: a + b)
list_n_words_groups.takeOrdered(10, key = lambda x: -x[1])


23/02/18 23:31:13 WARN TaskSetManager: Stage 2 contains a task of very large size (2023 KiB). The maximum recommended task size is 1000 KiB.
23/02/18 23:31:13 WARN TaskSetManager: Stage 3 contains a task of very large size (2023 KiB). The maximum recommended task size is 1000 KiB.


[(('I', 'have', 'no', 'doubt'), 12),
 (('have', 'no', 'doubt', 'that'), 12),
 (('to', 'me', 'to', 'be'), 10),
 (('in', 'front', 'of', 'the'), 10),
 (('beg', 'that', 'you', 'will'), 8),
 (('the', 'terms', 'of', 'this'), 8),
 (('up', 'and', 'down', 'the'), 7),
 (('the', 'corner', 'of', 'the'), 7),
 (('I', 'do', 'not', 'know'), 7),
 (('seemed', 'to', 'me', 'that'), 7)]

In [60]:
sc.stop()

In [73]:
from pyspark import SparkContext, SparkConf

sc = SparkContext("local", "Word Count")
txt = sc.textFile('Arthur.txt') # list of line of sentences
#n = 5 # n-gram
#n = 4 # n-gram
#n = 2 # n-gram
n=1
words = txt.flatMap(lambda line: n_grams(n , line)) # transforming sentences into separate words



ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Word Count, master=local) created by __init__ at /var/folders/ty/gwnmr_d57774fcnl_52r4y400000gn/T/ipykernel_6207/4216913568.py:3 

In [160]:
def check_Camel(word):
    if (len(word)>4 and word[0].isupper() and word[1:].islower())  :
        return True
    return False

In [165]:
words_list = [word[0] for word in words.collect() ]
rdd_nwords = sc.parallelize(words_list)
list_n_words = rdd_nwords.map(lambda word: ( word if(check_Camel(word)  )else" " , 1))
list_n_words.collect()
list_n_words_groups = list_n_words.reduceByKey(lambda a, b: a + b)

In [166]:
list_n_words_groups.takeOrdered(10, key = lambda x: -x[1])

[(' ', 76856),
 ('Holmes', 166),
 ('Holmes,', 105),
 ('There', 99),
 ('Sherlock', 77),
 ('Project', 45),
 ('Holmes.', 44),
 ('Watson,', 32),
 ('Gutenberg-tm', 31),
 ('Baker', 21)]